# Deep Reinforcement Learning

* [Deep Q Learning Nature Paper - Human-level control through deep reinforcement
learning](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf)
* [Playing Atari with Deep Reinforcement Learning](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf)


## Agent and Environment

Agent는 Environment(여기서는 Atari Game)과 연동을 합니다.<br>
각각의 time-step마다 Agent는 $ A = {1, ..., K} $중의 하나의 액션 $ a_t $를 선택하고, Environment로부터 화면 이미지 $ x_t \in \mathbb{R}^d $와 reward $ r_t $를 받습니다.<br>
하지만 하나의 게임 화면 이미지로만으로는 예를 들어 블록깨기에서 공이 어디 방향으로 가는지 알 수가 없습니다. <br>
따라서 $ x_t $를 시계열의 데이터로 받으며 일련의 actions과 observations 의 연속성이 됩니다.

$$ s_t = x_1, a_1, x_2, ..., a_{t-1}, x_t  $$

## Future Discounted Return 

Agent의 목표는 future reward를 최대치로 하는 actions을 선택하는 것입니다.<br>
여기서 future reward란 $ \gamma $ 배수만큼 (a factor of $ \gamma $ per time-step) discounted 되는 것을 의미합니다. <br>
쉽게 이야기해서 먼미래의 reward일수록, 더 적은 reward로 계산하겠다는 뜻입니다.

### $$ R_t = \sum^T_{t^{\prime} = t} \gamma^{t^{\prime} - t} r_{t^{\prime}} $$

$ T $는 게임이 끝나는 시점의 time-step을 의미합니다.



## Optimal action-value Function and Bellman Equation 

<span style="color:red">**Optimal action-value function $ Q^{*}(s, a) $**</span>이란 
policy를 따름으로서 얻을수 있는 maximum expected return으로 정의할수 있으며 다음의 공식과 같습니다.<br>
(policy $ \pi $는  state -> action 으로 연결시키는 매핑이라고 쉽게 생각할 수 있습니다. 다르게 말하면 distributions over actions)

$$ \begin{align}
Q^{*} (s, a) &= \max_{\pi} \mathbb{E} \left[ r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + ... \big|\ s_t =s, a_t = a, \pi \right] \\
&= \max_{\pi} \mathbb{E} \left[ R_t \big| \ s_t=s, a_t =a, \pi \right] 
\end{align} $$

Optimal Action-Value Function은 <span style="color:red">**Bellman Equation**</span>을 따릅니다. <br>
만약 optimal value $ Q^{*}(s^{\prime}, a^{\prime}) $ (여기서 $ s^{\prime} $는 next time-step의 state이고, $ a^{\prime} $는 모든 가능한 actions들을 말함) 의 값을 알고 있다면, <br>
expected value $ r + \gamma Q^{*}(s^{\prime}, a^{\prime}) $를 maximize 하는 action $ a^{\prime} $을 선택하는 것에 기초를 두고 있습니다.

$$ Q^{*} (s, a) = E_{s^{\prime} \sim \varepsilon} \left[ r + \gamma \max_{a^{\prime}} Q^{*} \left(s^{\prime}, a^{\prime}\right) \  \big| \ s, a \right] $$



## Nonlinear Approximator and Loss Function

기본적으로 많은 reinforcement learning algorithms들의 아이디어는 <br>
위의 Bellman Equation을 사용하여 iterative update로서 action-value function을 구하는 것입니다.<br>
다음은 Value Iteration같은 알고리즘같이 **iterative update**를 사용하는 공식. 

$$ Q_{i+1}(s, a) = 
\mathbb{E}_{s^{\prime}} \left[ r + \gamma \max_{a^{\prime}} Q^{*} \left( s^{\prime}, a^{\prime} \right) \big| \ s, a \right] $$

하지만 위의 공식은 각각의 sequence마다 action-value가 측정되며 일반화시키지 못합니다. <br>
따라서 **function approximator**를 사용하여 action-value function을 측정합니다.

$$ Q(s, a; \theta) \approx Q^{*} (s, a) $$

보통 Reinforcement Learning에서는 Linear function approximator를 사용하지만, <br>
Deep Q Learning에서는 Nonlinear function approximator인 뉴럴네트워크를 사용합니다.<br>
위의 공식에서 neural network function approximator로서 weights $ \theta $가 Q-network로 사용되었습니다.<br>
즉 Q-network는 parameters $ \theta $를 조정해가면서 학습이 진행됩니다.

Loss function은 mean-squared error를 사용합니다.<br>
이때 Bellman equation을 사용하게 되는데 target values $ r + \gamma \max_{a^{\prime}} Q^{*}\left(s^{\prime}, a^{\prime}\right) $ 이 부분을 approximate target values $ y = r + \gamma \max_{a^{\prime}} Q \left( s^{\prime}, a^{\prime}; \theta_{i-1} \right) $

# TODO!!!!!!!!!!!!!!!


### Experience Replay and Loss Function

게임을 진행하면서 학습을 할 경우 observation sequence같의 연관성(correlation)때문에 학습이 제대로 안 될수 있습니다.<br>
연관성을 끊어주는 방법으로 experience replay를 사용합니다.

먼저 Agent의 experiences $ e_t = (s_t, a_t, r_t, s_{t+1} )$를 각각의 time-step마다 data set $ D_t = \{ e_1, e_2, ..., e_t \} $에 저장합니다.<br>
학습시 Q-Learning updates를 uniformly random으로 꺼내진 experiences $ (s, a, r, s^{\prime}) \sim U(D) $ 통해 실행하게 됩니다.<br>
Q-Learning update는 다음의 loss function을 사용하게 됩니다.

$$ L_i(\theta_i) = \mathbb{E}_{s, a, r, s^{\prime}} \sim U(D) \left[ \left( r + \gamma \max_{a^{\prime}} Q\left(s^{\prime}, a^{\prime}; \theta_{i-1}\right) - Q\left( s, a; \theta_i \right) \right)^2 \right] $$

위의 공식을 Differentiate하면 다음과 같은 결과를 얻습니다.

$$ \nabla $$

$$ \nabla_{\theta_i} L_i (\theta_i) = \mathbb{E}_ $$

### Deep Q-Learning with Experience Replay Algorithm

<img src="./images/deep-q-learning-algorithm.png">
